# cuGraph Cheat Sheets sample code

(c) 2020 NVIDIA, Blazing SQL

Distributed under Apache License 2.0

### Imports

In [1]:
import cudf
import cugraph
import json 
import pandas as pd

### Create graph dataframes

We use the Star Wars social network graph prepared by Evelina Gabasova: Gabasova, E. (2016). Star Wars social network. DOI: https://doi.org/10.5281/zenodo.1411479.

In [2]:
interactions = json.loads(open('data/starwars-full-interactions.json', 'r').read())

edges = cudf.from_pandas(pd.DataFrame(interactions['links']))
nodes = cudf.from_pandas(pd.DataFrame(interactions['nodes'])).drop('colour', axis=1).reset_index()

g = cugraph.Graph()
g.from_cudf_edgelist(
    edges
    , source='source'
    , destination='target'
    , edge_attr='value'
    , renumber=True
)

---

# Graph community

---

#### cugraph.community.ecg.ecg()

In [3]:
cugraph.ecg(g, min_weight=0.05, ensemble_size=20)

,partition,vertex
0,0,54
1,8,53
2,6,69
3,2,73
4,0,20
...,...,...
104,0,23
105,3,107
106,7,22
107,3,108


#### cugraph.community.ktruss_subgraph.ktruss_subgraph()

In [4]:
cugraph.ktruss_subgraph(g, 3, use_weights=True).edges()

,src,dst
0,11,15
1,0,23
2,5,7
3,14,42
4,9,13
...,...,...
382,8,64
383,4,45
384,8,10
385,18,26


#### cugraph.community.leiden.leiden()

In [5]:
parts, modularity_score = cugraph.leiden(g, max_iter=10)

In [6]:
parts.head()

,partition,vertex
0,0,54
1,0,53
2,3,69
3,1,73
4,0,20


In [7]:
modularity_score

0.45214203000068665

#### cugraph.community.louvain.louvain()

In [8]:
parts, modularity_score = cugraph.louvain(g, max_iter=10)

In [9]:
parts.head()

,partition,vertex
0,0,54
1,0,53
2,3,69
3,1,73
4,0,20


In [10]:
modularity_score

0.45214203000068665

#### cugraph.community.subgraph_extraction.subgraph()

In [11]:
g_sub = cugraph.subgraph(g, cudf.Series([0,1,3]))

In [12]:
g_sub.nodes()

0    3
1    1
2    0
Name: 0, dtype: int64

In [13]:
g_sub.edges()

,src,dst
0,1,3
1,0,1
2,0,3


#### cugraph.community.triangle_count.triangles()

In [14]:
cugraph.triangles(g)

2013

#### cugraph.community.spectral_clustering.spectralBalancedCutClustering()

In [15]:
spectral = cugraph.spectralBalancedCutClustering(g, 5)

In [16]:
spectral

,cluster,vertex
0,2,88
1,0,72
2,2,87
3,4,5
4,2,25
...,...,...
104,0,31
105,2,105
106,3,98
107,4,12


#### cugraph.community.spectral_clustering.analyzeClustering_edge_cut()

In [17]:
spectral = cugraph.spectralBalancedCutClustering(g, 5)
cugraph.analyzeClustering_edge_cut(g, 5, spectral)

527.0

#### cugraph.community.spectral_clustering.analyzeClustering_modularity()

In [18]:
spectral = cugraph.spectralBalancedCutClustering(g, 5)
cugraph.analyzeClustering_modularity(g, 5, spectral)

0.1427346020936966

#### cugraph.community.spectral_clustering.analyzeClustering_ratio_cut()

In [19]:
spectral = cugraph.spectralBalancedCutClustering(g, 5)
cugraph.analyzeClustering_ratio_cut(g, 5, spectral)

31.196733474731445

#### cugraph.community.spectral_clustering.spectralModularityMaximizationClustering()

In [20]:
cugraph.spectralModularityMaximizationClustering(
    g
    , num_clusters=5
    , num_eigen_vects=3
    , evs_max_iter=200
    , kmean_tolerance=1e-06
    , kmean_max_iter=200
)

,cluster,vertex
0,4,88
1,4,72
2,4,87
3,2,5
4,1,25
...,...,...
104,1,31
105,1,105
106,1,98
107,1,12
